In [2]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

  ### DECISION TREE

In [4]:
df_casual = pd.read_pickle('datos/bikes_casual_estand.pkl')
df_casual.head()

,estacion,año,mes,festivo,dia_semana,dia_laboral,tiempo,cliente_casual,temperatura,velocidad_viento
1,1.0,1.0,1.0,1,1.5,1.5,3,131.0,-1.636215,0.744965
2,1.0,1.0,1.5,3,1.0,1.0,3,120.0,-1.616347,-0.390073
3,1.0,1.0,3.5,3,1.0,1.0,4,108.0,-1.469045,-0.046981
4,2.0,1.0,3.0,3,1.5,1.5,4,82.0,-1.592588,-1.302236
5,2.0,1.0,3.5,3,1.0,1.0,4,88.0,-1.635352,-0.281358


In [5]:
X = df_casual.drop("cliente_casual", axis = 1)
y = df_casual["cliente_casual"]